In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from tqdm import tqdm 

In [ ]:
IMAGE_SIZE = 64

transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [ ]:
BATCH_SIZE = 32

train_dataset = datasets.ImageFolder(root="./data/CIFAKE/train", transform=transform)
test_dataset = datasets.ImageFolder(root="./data/CIFAKE/test", transform=transform)

val_ratio = 0.2  
train_size = int((1 - val_ratio) * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [37]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Freeze earlier layers (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for binary classification with sigmoid
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1)
)

model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
# Training loop
# Add early stopping
# Create list of train and validation losses/accuracies

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch (Training) {epoch+1}/{EPOCHS}", leave = False):
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # reshape to (batch, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        preds = torch.sigmoid(outputs) >= 0.5
        correct += (preds == labels.bool()).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / len(train_loader.dataset)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc=f"Epoch (Validation) {epoch+1}/{EPOCHS}", leave = False):
            val_images, val_labels = val_images.to(device), val_labels.to(device).float().unsqueeze(1)
            val_outputs = model(val_images)
            val_preds = torch.sigmoid(val_outputs) >= 0.5
            val_correct += (val_preds == val_labels.bool()).sum().item()
            val_total += val_labels.size(0)

    val_acc = val_correct / val_total
    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Acc: {val_acc:.4f}")

torch.save(model, "./models/detection_resnet18.pth")

Epoch 1/10: 100%|██████████| 2500/2500 [00:52<00:00, 47.49it/s]


Epoch 1/10, Train Loss: 0.4362, Train Acc: 0.7981, Val Acc: 0.8282


Epoch 2/10: 100%|██████████| 2500/2500 [01:02<00:00, 40.00it/s]


Epoch 2/10, Train Loss: 0.4134, Train Acc: 0.8123, Val Acc: 0.8188


Epoch 3/10: 100%|██████████| 2500/2500 [01:01<00:00, 40.49it/s]


...

Epoch 8/10, Train Loss: 0.4103, Train Acc: 0.8150, Val Acc: 0.8191


Epoch 9/10: 100%|██████████| 2500/2500 [01:02<00:00, 40.10it/s]


Epoch 9/10, Train Loss: 0.4120, Train Acc: 0.8140, Val Acc: 0.8287


Epoch 10/10: 100%|██████████| 2500/2500 [01:01<00:00, 40.42it/s]


Epoch 10/10, Train Loss: 0.4147, Train Acc: 0.8111, Val Acc: 0.8319


In [40]:
threshold = 0.7 

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs >= threshold).int() 

        correct += (preds == labels.int()).sum().item()
        total += labels.size(0)

test_accuracy = correct / total
print(f"Test Accuracy (threshold={threshold}): {test_accuracy:.4f}")

Test Accuracy (threshold=0.7): 0.7984
